In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn import svm
import warnings

warnings.filterwarnings("ignore")
from flaml import logger, logging

logger.setLevel(logging.WARNING)
import random

imp = pd.read_csv("Imputed_scaled.csv")
miss = pd.read_csv("Missingness_0_scaled.csv")
smote = pd.read_csv("Smote_scaled.csv")
TEST = pd.read_csv("unlabeled.csv")

cols = imp.columns
cols = cols.drop(cols[0])

test_X = TEST[cols]
testing_X = test_X.drop("hospital_death", axis=1)

imp = imp[cols]
miss = miss.drop(miss.columns[0], axis=1)
smote = smote.drop(smote.columns[0], axis=1)

miss_X = miss.drop("hospital_death", axis=1)
miss_y = miss["hospital_death"]
random.seed(10)
miss_x_train, miss_x_test, miss_y_train, miss_y_test = train_test_split(
    miss_X, miss_y, test_size=0.3, random_state=23
)

from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import uniform

dists = {
    "C": uniform(loc=0, scale=3),
    "gamma": uniform(loc=0, scale=(3 / 171)),  # number of features
}

rbf_cv = RandomizedSearchCV(
    svm.SVC(random_state=13, kernel="rbf"),
    dists,
    n_iter=10,
    n_jobs=-1,
    scoring=["accuracy", "f1", "precision", "recall"],
    refit="accuracy",
    cv=5,
    random_state=14,
    verbose=0,
)

rbf_cv.fit(miss_x_train, miss_y_train)

rbf_cv_pred = rbf_cv.predict(miss_x_test)
rbf_cv_sc = rbf_cv.score(miss_x_test, miss_y_test)
rbf_cv_results = pd.DataFrame(rbf_cv.cv_results_)

print(rbf_cv_sc)

print(rbf_cv_results)


0.9302900341644254
   mean_fit_time  std_fit_time  mean_score_time  std_score_time   param_C  \
0     993.770919      5.569846        38.108131        0.224655   1.54183   
1      99.811231      1.294084        22.572489        0.177981  2.611283   
2     915.404325      9.109735        41.801132        0.381763  0.929208   
3     503.473458      8.912176        27.109128        0.565679   1.53935   
4     373.475143      3.756704        24.931230        0.445312    1.6176   
5     693.079203      4.938309        30.599938        0.300551  2.419444   
6      91.876734      1.473987        19.660790        0.845626  1.616667   
7     388.657130      4.636584        24.616435        0.243811  2.019457   
8     796.248815      4.202659        27.971120        2.434491  2.797673   
9    1021.307385      6.143257        33.622133        0.373861  2.257257   

  param_gamma                                             params  \
0    0.013564  {'C': 1.5418300313180708, 'gamma': 0.013564299... 

In [3]:
pd.set_option('display.max_columns', None)
rbf_cv_results.loc[rbf_cv_results['rank_test_accuracy']==1]

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_C,param_gamma,params,split0_test_accuracy,split1_test_accuracy,split2_test_accuracy,split3_test_accuracy,split4_test_accuracy,mean_test_accuracy,std_test_accuracy,rank_test_accuracy,split0_test_f1,split1_test_f1,split2_test_f1,split3_test_f1,split4_test_f1,mean_test_f1,std_test_f1,rank_test_f1,split0_test_precision,split1_test_precision,split2_test_precision,split3_test_precision,split4_test_precision,mean_test_precision,std_test_precision,rank_test_precision,split0_test_recall,split1_test_recall,split2_test_recall,split3_test_recall,split4_test_recall,mean_test_recall,std_test_recall,rank_test_recall
7,388.65713,4.636584,24.616435,0.243811,2.019457,0.003685,"{'C': 2.019457434541576, 'gamma': 0.0036846361...",0.927181,0.92796,0.926636,0.929439,0.927487,0.92774,0.000952,1,0.361775,0.355401,0.338483,0.378601,0.345748,0.356002,0.013831,3,0.744382,0.782209,0.765079,0.793103,0.783439,0.773643,0.017191,4,0.238954,0.229937,0.217313,0.248649,0.221821,0.231335,0.011374,3


In [ ]:

from sklearn.impute import SimpleImputer

imputer = SimpleImputer(strategy="most_frequent")  # another: median
test_trunc_x_imp = imputer.fit_transform(testing_X)

rbf_cv_pred_test = rbf_cv.predict(test_trunc_x_imp)
rbf_cv_results_test = pd.DataFrame(rbf_cv.cv_results_)

rbf_cv_pred_test0 = rbf_cv.predict(testing_X.fillna(0))
rbf_cv_results_test0 = pd.DataFrame(rbf_cv.cv_results_)